In [2]:
import bs4
import pandas as pd
from urllib import request
import re

In [3]:
link = "https://www.senscritique.com/"
list_class=["https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2010/1702619",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2011/1702611",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2012/165128",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2013/173210",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2014/367139",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2015/703344",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2016/1172216",
           "https://www.senscritique.com/liste/Meilleures_series_2017/1586999",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2018/1757801",
           "https://www.senscritique.com/top/resultats/Les_meilleures_series_de_2019/2301804"]

In [4]:
def text_strip(t):
    if t is not None :
        return(t.text.strip())
    else :
        return("Inconnu")
 

In [5]:
#1e fonction destinée à extraire les liens des pages SensCritique des séries d'un classement
def links_series(link_class):
    request_text = request.urlopen(link_class).read()
    page = bs4.BeautifulSoup(request_text, "lxml")
    series = page.findAll('li', {"class" : "elpo-item"})
    links_dic = {}
    for li in series :
        name = li.select('a[class*="elco-anchor"]')
        if name is [] :
            pass
        else :
            name = name[0].text.strip()
            link_serie = link + li.find('div', {"class" : "elpo-column"})['data-sc-link']
            grade = li.find('div', {"class" : "d-posterRating"})
            links_dic[name] = link_serie
    return(links_dic)
#2e fonction qui extrait les notes, auteurs, genres, nombres de saisons, plateforme de diffusion, la durée et la date de sortie des séries
def extract_data_series(links_dic):
    data = {}    
    columns = ['Note', 'Auteur', 'Genre', 'Nombre de saisons', 'Plateforme', 'Durée', 'Date de sortie']
    for name, l in links_dic.items():
        request_text = request.urlopen(l).read()
        page = bs4.BeautifulSoup(request_text, "lxml")
        section = page.find('section', {"class" : "pvi-productDetails"})
        if not section is None :
            l_li = section.findAll('li', {"class" : "pvi-productDetails-item"})
            text = [ele.get_text().strip() for ele in l_li]
            text = [re.sub('\[ (\d+) \] ?', '', ele) for ele in text]
            text = [ele.replace('\n', '') for ele in text]
            text = [ele.replace('\t', '') for ele in text]
            if len(page.select('span[class*="pvi-scrating-value"]')) == 0 :
                grades = "Inconnu"
            else : 
                grades = page.select('span[class*="pvi-scrating-value"]')[0].text.strip()
            if len(text) > 6 :
                text = text[6:]
            else :
                text = text + ["Inconnu" for k in range(6-len(text))]

            data[name] = [grades] + text

    df = pd.DataFrame.from_dict(data, orient = 'index', columns = columns)
    return(df)


In [7]:
series_info= pd.concat(list_df)
series_info

,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5 saisons(en cours),BBC One,1 h 30 min,25 juillet 2010
The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10 saisons(en cours),AMC,43 min,11 octobre 2010
Boardwalk Empire,7.6,Série de Terence Winter,Drame,5 saisons(terminée),HBO,1 h,19 septembre 2010
Luther,7.6,Série de Neil Cross,Policier et drame,5 saisons(en cours),BBC One,1 h,4 mai 2010
Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1 saison(terminée),HBO,50 min,14 mars 2010
Downton Abbey,7.6,Série de Julian Fellowes,Drame,6 saisons(terminée),ITV,1 h,26 septembre 2010
Spartacus,6.7,Série de Steven S. DeKnight,"Action, drame et historique",3 saisons(terminée),Starz,1 h,22 janvier 2010
Game of Thrones,8.2,Série de David Benioff et D.B. Weiss,"Aventure, drame et fantasy",8 saisons(terminée),HBO,52 min,5 décembre 2010
Treme,8.1,Série de David Simon et Eric Overmyer,Drame,4 saisons(terminée),HBO,1 h,11 avril 2010
Justified,7.2,Série de Graham Yost,"Policier, drame et action",6 saisons(terminée),FX,45 min,16 mars 2010


In [ ]:
series_info.to_csv('C:/Users/Jean-Baptiste/OneDrive/ENSAE/projet_python_data.csv')